In [1]:
!pip install snntorch brevitas --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.4/411.4 kB 9.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# @title Default title text
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import csv
import itertools
import collections

import pywt
from scipy import stats

# from utils import denoise

plt.rcParams["figure.figsize"] = (30,6)
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['lines.color'] = 'b'
plt.rcParams['axes.grid'] = True




class Dataset:
    def __init__(self):
        self.dataset = None
        self.data_path = None


    def denoise(data):
        w = pywt.Wavelet('sym4')
        maxlev = pywt.dwt_max_level(len(data), w.dec_len)
        threshold = 0.04 # Threshold for filtering

        coeffs = pywt.wavedec(data, 'sym4', level=maxlev)
        for i in range(1, len(coeffs)):
            coeffs[i] = pywt.threshold(coeffs[i], threshold*max(coeffs[i]))

        datarec = pywt.waverec(coeffs, 'sym4')

        return datarec

    def load_dataset(self, data_path):

        print("Loading dataset from: ", data_path)

        records = list()
        annotations = list()

        for file in os.listdir(data_path):
            if file.endswith(".csv"):
                records.append(data_path+file)
            elif file.endswith(".txt"):
                annotations.append(data_path+file)

        records.sort()
        annotations.sort()

        # print(records[0], annotations[0])

        return records, annotations




    def preprocess(self,records,annotations):
        maximum_counting = 10000
        window_size = 180
        classes = ['N', 'L', 'R', 'A', 'V']
        n_classes = 5
        count_classes = [0]*5
        X = list()
        y = list()
        for r in range(0,len(records)):
            signals = []

            with open(records[r], 'rt') as csvfile:
                spamreader = csv.reader(csvfile, delimiter=',', quotechar='|') # read CSV file\
                row_index = -1
                for row in spamreader:
                    if(row_index >= 0):
                        signals.insert(row_index, int(row[1]))
                    row_index += 1

            # # Plot an example to the signals
            # if r == 6:
            #     # Plot each patient's signal
            #     plt.title(records[6] + " Wave")
            #     plt.plot(signals[0:700])
            #     plt.show()
            def denoise(data):
                w = pywt.Wavelet('sym4')
                maxlev = pywt.dwt_max_level(len(data), w.dec_len)
                threshold = 0.04 # Threshold for filtering

                coeffs = pywt.wavedec(data, 'sym4', level=maxlev)
                for i in range(1, len(coeffs)):
                    coeffs[i] = pywt.threshold(coeffs[i], threshold*max(coeffs[i]))

                datarec = pywt.waverec(coeffs, 'sym4')

                return datarec
            signals = denoise(signals)
            # # Plot an example to the signals
            # if r == 6:
            #     # Plot each patient's signal
            #     plt.title(records[6] + " wave after denoised")
            #     plt.plot(signals[0:700])
            #     plt.show()

            # signals = stats.zscore(signals)
            # # Plot an example to the signals
            # if r == 6:
            #     # Plot each patient's signal
            #     plt.title(records[6] + " wave after z-score normalization ")
            #     plt.plot(signals[0:700])
            #     plt.show()

            # Read anotations: R position and Arrhythmia class
            example_beat_printed = False
            with open(annotations[r], 'r') as fileID:
                data = fileID.readlines()
                beat = list()

                for d in range(1, len(data)): # 0 index is Chart Head
                        splitted = data[d].split(' ')
                        splitted = filter(None, splitted)
                        next(splitted) # Time... Clipping
                        pos = int(next(splitted)) # Sample ID
                        arrhythmia_type = next(splitted) # Type
                        if(arrhythmia_type in classes):
                            arrhythmia_index = classes.index(arrhythmia_type)
                    # if count_classes[arrhythmia_index] > maximum_counting: # avoid overfitting
                            #    pass
                        #else:
                            count_classes[arrhythmia_index] += 1
                            if(window_size <= pos and pos < (len(signals) - window_size)):
                                beat = signals[pos-window_size:pos+window_size]     ## REPLACE WITH R-PEAK DETECTION
                                # Plot an example to a beat
                                # if r == 6 and not example_beat_printed:
                                #     plt.title("A Beat from " + records[6] + " Wave")
                                #     plt.plot(beat)
                                #     plt.show()
                                #     example_beat_printed = True

                                X.append(beat)
                                y.append(arrhythmia_index)

        return X, y

In [4]:
# @title Default title text
import pywt
import matplotlib.pyplot as plt

# from dataset import Dataset

plt.rcParams["figure.figsize"] = (30,6)
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['lines.color'] = 'b'
plt.rcParams['axes.grid'] = True


def plot_signals(signals, title):

    plt.title("ECG signal: " + title)
    plt.plot(signals)
    plt.show()

In [5]:
dataset = Dataset()
data_path = '/content/drive/MyDrive/dataset/'

records, annotations = dataset.load_dataset(data_path)

X = []
y = []
X, y = dataset.preprocess(records, annotations)

Loading dataset from:  /content/drive/MyDrive/dataset/


In [15]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, X, y,transform=None):
        self.data = torch.tensor(X, dtype=torch.float32)
        self.targets = torch.tensor(y, dtype=torch.long)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]


In [16]:
from sklearn.model_selection import train_test_split

import numpy as np

min_value = np.min(X)
max_value = np.max(X)

normalised_X = (X - min_value) / (max_value - min_value)

X_train, X_test, y_train, y_test = train_test_split(normalised_X, y, random_state=104, train_size=0.8, shuffle=True)

In [17]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

batch_size = 128
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Assuming you have loaded your data and labels into train_loader and test_loader

class ECGNet(nn.Module):
    def __init__(self, num_classes=5):
        super(ECGNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5)

        # Calculate the output size after convolutions and pooling
        self.conv_out_size = self.calculate_conv_output_size()

        self.fc1 = nn.Linear(64 * self.conv_out_size, 100)
        self.fc2 = nn.Linear(100, num_classes)

    def calculate_conv_output_size(self):
        # Calculate the output size after convolutions and pooling
        x = torch.randn(1, 1, 360)  # Example input
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        return x.size(2)

    def forward(self, x):
        x = torch.relu(self.pool(self.conv1(x)))
        x = torch.relu(self.pool(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the output for fully connected layers
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Initialize the model, loss function, and optimizer
model = ECGNet(num_classes=5)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        outputs = model(inputs.unsqueeze(1))  # Add 1 channel dimension for the input
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs.unsqueeze(1))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy on the test data: {}%'.format(100 * correct / total))


Epoch [1/10], Step [100/626], Loss: 0.9042
Epoch [1/10], Step [200/626], Loss: 0.9640
Epoch [1/10], Step [300/626], Loss: 0.5959
Epoch [1/10], Step [400/626], Loss: 0.4928
Epoch [1/10], Step [500/626], Loss: 0.3924
Epoch [1/10], Step [600/626], Loss: 0.4230
Epoch [2/10], Step [100/626], Loss: 0.2658
Epoch [2/10], Step [200/626], Loss: 0.2076
Epoch [2/10], Step [300/626], Loss: 0.3716
Epoch [2/10], Step [400/626], Loss: 0.4192
Epoch [2/10], Step [500/626], Loss: 0.2190
Epoch [2/10], Step [600/626], Loss: 0.3198
Epoch [3/10], Step [100/626], Loss: 0.1128
Epoch [3/10], Step [200/626], Loss: 0.1747
Epoch [3/10], Step [300/626], Loss: 0.1622
Epoch [3/10], Step [400/626], Loss: 0.1542
Epoch [3/10], Step [500/626], Loss: 0.1449
Epoch [3/10], Step [600/626], Loss: 0.2160
Epoch [4/10], Step [100/626], Loss: 0.0714
Epoch [4/10], Step [200/626], Loss: 0.1472
Epoch [4/10], Step [300/626], Loss: 0.1206
Epoch [4/10], Step [400/626], Loss: 0.1134
Epoch [4/10], Step [500/626], Loss: 0.1547
Epoch [4/10